# Erster Agent

Es ist an der Zeit einen Agenten zu erstellen. Und zwar werden wir einen Agenten mit zwei-drei Tools erstellen. 
Als erstes Tool möchten wir dem Agenten Zugriff auf Wikipedia geben, damit der Agent mithilfe der Wikipedia-Artikel fundierte Antworten geben kann.
Als zweites Tool bekommt der die DuckDuckGo-Funktion.



In [ ]:
!pip install langchain langchain_openai langchain_community python-dotenv duckduckgo-search wikipedia

In [ ]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5)

Wir legen uns bekannten Tools an...

In [ ]:
search = DuckDuckGoSearchResults()

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

Jetzt ist es an der Zeit den Agenten zu erstellen. Hierfür werden wir eine Hilfsfunktion nutzen: _create_tool_calling_agent_. Diese Funktion kombiniert Modell, Prompt und Tools zu einem Agenten, der in der Lage ist die Tools auszuführen.

Das Prompt-Template sieht etwas wild aus. Erstmal bekommt es natürlich einen Systemprompt, der unseren Agenten eine Rolle gibt. Falls wir den Agenten in einem Chat nutzen möchten, definieren wir optional einen Platzhalter für die _chat\_history_. Natürlich muss auch die Frage des Benutzers einen Platz haben... dafür fügen wir _HumanMessagePromptTemplate_ ein.

Der letzte Platzhalter ist allerdings neu: _agent\_scratchpad_. Wozu das gut ist, sehen wir im nächsten Abschnitt etwas besser. Auf jeden Fall ist das eine Art Notizblock für den Agenten, wo er seine Zwischenergebnisse, z.B. Tool-Ausgaben, notieren kann. D.h. diese Daten sind im Kontext des Agenten.

In [ ]:
system_template = "Du bist ein hilfreicher Assistent, der in der Lage ist die notwendigen Fakten im Internet zu suchen oder bei Wikipedia nachzuschlagen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_template)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt_template)

Jetzt wo wir den Agenten haben, brauchen wir noch einen _AgentExecutor_. Der _AgentExecutor_ ist praktisch die Laufzeitumgebung, die dafür sorgt, dass der Agent ausgeführt wird, die Tools, die er braucht ausführt und desssen Ergebnisse an den Agenten übergibt.

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Ok lass uns den Agenten mal ausprobieren. Stellen wir eine Wissensfrage und schauen wir nach, ob der Agent sich dazu entschließt ein Tool zu nutzen. Denn theoretisch gehört die Frage nach dem Geburtstag von Olaf Scholz mit Sicherheit zu den Trainingsdaten. 

In [ ]:
result = agent_executor.invoke({"input": "Guten Tag! Wann wurde Budeskanuzler Olaf Scholz geboren?"})

result

Jetzt versuchen wir den Agenten dazu zu bringen, das Such-Tool zu nutzen.

In [ ]:
result = agent_executor.invoke({"input": "Wann findet das Münchner Oktoberfest 2024 statt?"})

result

Hier geht es zum nächsten Abschnitt: ReAct Agent: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ki-zeitalter/ai-agent-workshop/blob/main/07_react_agent.ipynb)